# 2022.07.14 Version1: pcap, pcap-ng --> .csv

# 実行:File Path -> Specific Character -> Run

In [20]:
#Import
import dpkt 
import csv
import glob
import numpy as np
import os

TH_SYN = 0x02
TH_ACK = 0x10
OUTFILE= "2016-09-30.csv"
#df = pd.DataFrame(datalist, columns=metaif)
metaif = ["IPID", "TTL","SRCIP", "DSTIP", "SPORT", "DPORT", "TSEQ", "WINDS"]

#file1 = glob.glob("/home/pison/D1/cluster/pcap/e70*.pcap")
#files = file1 + file2
#files = glob.glob("/home/pison/D1/cluster/pcap/pison*.pcap")
#file2 = glob.glob("/home/pison/D1/cluster/pcap/e435*.pcap")

file2 = glob.glob("/data/darknet/2016/2016-09-30.pcap.zst")
files = sorted(file2)[:1]
print(files)

['/data/darknet/2016/2016-09-30.pcap.zst']


In [24]:
#Make Data from each pcap file
for file in files:
    datalist = []
    fp = open(file, "rb")
    sig = (fp.read(4)).hex()
    fp.seek(0)
    
    if sig == 'd4c3b2a1':
        packets = dpkt.pcap.Reader(fp)
    elif sig == '0a0d0d0a':
        packets = dpkt.pcapng.Reader(fp)
        
    #Packets を処理する
    for ts, buf in packets:
        try: 
            eth = dpkt.ethernet.Ethernet(buf)
        except:
            print("Fail parse Frame")

        #Verify: IP Packet
        if type(eth.data) == dpkt.ip.IP:
            ip = eth.data
            srcip 	= int.from_bytes(ip.src, "big")
            dstip	= int.from_bytes(ip.dst, "big")
            ipid 	= ip.id
            ipttl 	= ip.ttl

            #Verify: TCP packet
            if ip.p == dpkt.ip.IP_PROTO_TCP:
                tcp 	= ip.data
                try:
                    tsport 	= tcp.sport
                    tdport 	= tcp.dport
                    tseq 	= tcp.seq
                    twin 	= tcp.win
                    if tcp.flags == TH_SYN and tcp.flags != TH_ACK:
                        #if tcp.seq == dstip:
                        phrase = [ipid, ipttl, srcip, dstip, tsport, tdport, tseq, twin]
                        datalist.append(phrase)
                except:
                    continue
    filename = os.path.basename(file)
    title = filename.ljust(5, " ")+ ":" + str(len(datalist))
    print(title)
    outfile = str(filename) + ".csv"
    
    with open(outfile, "w") as ofile:
        write = csv.writer(ofile)
        write.writerow(metaif)
        write.writerows(datalist)

2016-09-30.pcap.zst:0


In [22]:
# Error 発生している